<a href="https://colab.research.google.com/github/leyli16/HousingPricePrediction/blob/fema_cleaning/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from string import ascii_letters
import matplotlib.pyplot as plt
import datetime as dt
import requests
from lxml import html
import math
import json
import re
import os

# 1. County Demographics Data Cleaning and Wrangling


## 1.1 Loading County Demogrphics Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("glozab/county-level-us-demographic-data-1990-2020")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/glozab/county-level-us-demographic-data-1990-2020/versions/1


In [ ]:
files = os.listdir(path)
print("Files in dataset:", files)

Files in dataset: ['county_demographics.csv']


In [ ]:
file_path = os.path.join(path, 'county_demographics.csv')
county_demographics_df = pd.read_csv(file_path)

print(county_demographics_df.head())

   year  fips  population  w_population  b_population  o_population  \
0  1990  1025       27289         15579         11643            35   
1  1990  1031       40293         32869          6950           160   
2  1990  1041       13598         10068          3516            11   
3  1990  1053       35526         24377         10050          1045   
4  1990  1101      209537        119702         87856           415   

   nh_population  hi_population  na_population  male_population  ...  \
0          27196             93              0            13052  ...   
1          39831            462              0            19673  ...   
2          13576             22              0             6421  ...   
3          35378            148              0            17454  ...   
4         207933           1604              0            98854  ...   

   age9_population_ratio  age10_population_ratio  age11_population_ratio  \
0                0.06263                 0.05552                

## 1.2 County Demographics Dataset Cleaning

In [ ]:
county_demographics_df.dtypes

,0
year,int64
fips,int64
population,int64
w_population,int64
b_population,int64
o_population,int64
nh_population,int64
hi_population,int64
na_population,int64
male_population,int64


In [ ]:
print(county_demographics_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97287 non-null  int64  
 1   fips                     97287 non-null  int64  
 2   population               97287 non-null  int64  
 3   w_population             97287 non-null  int64  
 4   b_population             97287 non-null  int64  
 5   o_population             97287 non-null  int64  
 6   nh_population            97287 non-null  int64  
 7   hi_population            97287 non-null  int64  
 8   na_population            97287 non-null  int64  
 9   male_population          97287 non-null  int64  
 10  female_population        97287 non-null  int64  
 11  age0_population          97287 non-null  int64  
 12  age1_population          97287 non-null  int64  
 13  age2_population          97287 non-null  int64  
 14  age3_population       

In [ ]:
print(county_demographics_df.isna().sum().sort_values(ascending=False))

year                       0
fips                       0
population                 0
w_population               0
b_population               0
o_population               0
nh_population              0
hi_population              0
na_population              0
male_population            0
female_population          0
age0_population            0
age1_population            0
age2_population            0
age3_population            0
age4_population            0
age5_population            0
age6_population            0
age7_population            0
age8_population            0
age9_population            0
age10_population           0
age11_population           0
age12_population           0
age13_population           0
age14_population           0
age15_population           0
age16_population           0
age17_population           0
age18_population           0
w_population_ratio         0
b_population_ratio         0
o_population_ratio         0
nh_population_ratio        0
hi_population_

### Rename Columns for Consistency

In [ ]:
county_demographics_df.columns = county_demographics_df.columns.str.strip().str.lower().str.replace(' ', '_')

 ### Handle Duplicates

In [ ]:
county_demographics_df = county_demographics_df.drop_duplicates()
county_demographics_df = county_demographics_df.dropna()
county_demographics_df.head()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age9_population_ratio,age10_population_ratio,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.06263,0.05552,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.07455,0.06031,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.06582,0.05354,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.06820,0.05694,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.06966,0.05219,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145


In [ ]:
county_demographics_df.describe()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age9_population_ratio,age10_population_ratio,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio
count,97287.000000,97287.000000,9.728700e+04,9.728700e+04,9.728700e+04,97287.000000,9.728700e+04,9.728700e+04,97287.0,9.728700e+04,...,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000
mean,2005.009436,30412.859950,9.365957e+04,7.529841e+04,1.252406e+04,1129.919044,8.009754e+04,1.356203e+04,0.0,4.598061e+04,...,0.068098,0.067555,0.064839,0.060595,0.054948,0.048050,0.039687,0.030821,0.021798,0.020099
std,8.942825,15147.842995,3.034699e+05,2.259523e+05,5.716518e+04,4815.437070,2.171562e+05,1.058393e+05,0.0,1.489240e+05,...,0.011292,0.010417,0.011904,0.014153,0.014746,0.013994,0.011871,0.009476,0.007503,0.008808
min,1990.000000,1001.000000,5.500000e+01,2.900000e+01,0.000000e+00,0.000000,4.900000e+01,0.000000e+00,0.0,2.900000e+01,...,0.008400,0.013420,0.009150,0.007710,0.005320,0.004840,0.002570,0.000000,0.000000,0.000000
25%,1997.000000,18183.000000,1.094800e+04,9.225500e+03,9.500000e+01,40.000000,1.011600e+04,1.710000e+02,0.0,5.445000e+03,...,0.059970,0.060400,0.055950,0.048960,0.044160,0.038620,0.032010,0.024680,0.016890,0.014210
50%,2005.000000,29179.000000,2.490200e+04,2.166100e+04,8.000000e+02,136.000000,2.327000e+04,6.350000e+02,0.0,1.231900e+04,...,0.068510,0.067780,0.065460,0.060580,0.052770,0.046150,0.038470,0.029950,0.021000,0.018590
75%,2013.000000,45083.000000,6.347900e+04,5.624700e+04,5.440000e+03,550.000000,5.989000e+04,3.148000e+03,0.0,3.135050e+04,...,0.075770,0.074500,0.073120,0.071100,0.064605,0.055630,0.045940,0.035970,0.025670,0.024350
max,2020.000000,99999.000000,1.009486e+07,7.301940e+06,1.437609e+06,172454.000000,5.508975e+06,4.868179e+06,0.0,4.975410e+06,...,0.160000,0.155960,0.153010,0.258060,0.181820,0.183910,0.183580,0.143400,0.144440,0.195400


## 1.2 Loading Fips Code DataSet




In [ ]:
from google.colab import files
uploaded = files.upload()
fips_df = pd.read_csv('state_and_county_fips_master.csv')
fips_df.head()

Saving state_and_county_fips_master.csv to state_and_county_fips_master.csv


,fips,name,state
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga County,AL
3,1003,Baldwin County,AL
4,1005,Barbour County,AL


In [ ]:
fips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   fips    3195 non-null   int64 
 1   name    3195 non-null   object
 2   state   3143 non-null   object
dtypes: int64(1), object(2)
memory usage: 75.0+ KB


## 1.3 Merging the county_demographics_df with fips_df

In [ ]:
county_demographics_merged_df = county_demographics_df.merge(
    fips_df,
    on='fips',
    how='left'
)
county_demographics_merged_df.head()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio,name,state
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319,Clarke County,AL
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258,Coffee County,AL
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839,Crenshaw County,AL
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368,Escambia County,AL
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145,Montgomery County,AL


### Sanity Check After Merging

In [ ]:
print(county_demographics_merged_df.columns)         # See all columns
print(county_demographics_merged_df[['fips', 'name']].head())  # Confirm correct matches
print(county_demographics_merged_df.isna().sum())    # Check for unmatched FIPS

Index(['year', 'fips', 'population', 'w_population', 'b_population',
       'o_population', 'nh_population', 'hi_population', 'na_population',
       'male_population', 'female_population', 'age0_population',
       'age1_population', 'age2_population', 'age3_population',
       'age4_population', 'age5_population', 'age6_population',
       'age7_population', 'age8_population', 'age9_population',
       'age10_population', 'age11_population', 'age12_population',
       'age13_population', 'age14_population', 'age15_population',
       'age16_population', 'age17_population', 'age18_population',
       'w_population_ratio', 'b_population_ratio', 'o_population_ratio',
       'nh_population_ratio', 'hi_population_ratio', 'na_population_ratio',
       'male_population_ratio', 'female_population_ratio',
       'age0_population_ratio', 'age1_population_ratio',
       'age2_population_ratio', 'age3_population_ratio',
       'age4_population_ratio', 'age5_population_ratio',
       'age6_popula

## 1.4 Cleaning the Merged Data

Checking for unmatched FIPS — that output tells us 174 rows in the county_demographics_df didn't find a match in fips_df, because name, state, and any other info from fips_df came back as NaN.

In [ ]:
unmatched = county_demographics_merged_df[county_demographics_merged_df['name'].isna()]
print(unmatched['fips'].unique()[:10])

[ 2910  4910 51917  8911  2201  8912  8913  2280  2232  2010]


In [ ]:
county_demographics_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97287 non-null  int64  
 1   fips                     97287 non-null  int64  
 2   population               97287 non-null  int64  
 3   w_population             97287 non-null  int64  
 4   b_population             97287 non-null  int64  
 5   o_population             97287 non-null  int64  
 6   nh_population            97287 non-null  int64  
 7   hi_population            97287 non-null  int64  
 8   na_population            97287 non-null  int64  
 9   male_population          97287 non-null  int64  
 10  female_population        97287 non-null  int64  
 11  age0_population          97287 non-null  int64  
 12  age1_population          97287 non-null  int64  
 13  age2_population          97287 non-null  int64  
 14  age3_population       

Drop the unmatched rows.

In [ ]:
county_demographics_merged_df = county_demographics_merged_df[county_demographics_merged_df['name'].notna()].reset_index(drop=True)
county_demographics_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97113 entries, 0 to 97112
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97113 non-null  int64  
 1   fips                     97113 non-null  int64  
 2   population               97113 non-null  int64  
 3   w_population             97113 non-null  int64  
 4   b_population             97113 non-null  int64  
 5   o_population             97113 non-null  int64  
 6   nh_population            97113 non-null  int64  
 7   hi_population            97113 non-null  int64  
 8   na_population            97113 non-null  int64  
 9   male_population          97113 non-null  int64  
 10  female_population        97113 non-null  int64  
 11  age0_population          97113 non-null  int64  
 12  age1_population          97113 non-null  int64  
 13  age2_population          97113 non-null  int64  
 14  age3_population       

In [ ]:
county_demographics_merged_df = county_demographics_merged_df.rename(columns={'name': 'county'})
county_demographics_merged_df = county_demographics_merged_df.dropna()
county_demographics_merged_df = county_demographics_merged_df.drop_duplicates()
county_demographics_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97113 entries, 0 to 97112
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97113 non-null  int64  
 1   fips                     97113 non-null  int64  
 2   population               97113 non-null  int64  
 3   w_population             97113 non-null  int64  
 4   b_population             97113 non-null  int64  
 5   o_population             97113 non-null  int64  
 6   nh_population            97113 non-null  int64  
 7   hi_population            97113 non-null  int64  
 8   na_population            97113 non-null  int64  
 9   male_population          97113 non-null  int64  
 10  female_population        97113 non-null  int64  
 11  age0_population          97113 non-null  int64  
 12  age1_population          97113 non-null  int64  
 13  age2_population          97113 non-null  int64  
 14  age3_population       

In [ ]:
county_demographics_merged_df.head()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio,county,state
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319,Clarke County,AL
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258,Coffee County,AL
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839,Crenshaw County,AL
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368,Escambia County,AL
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145,Montgomery County,AL


# 2. FEMA Disaster Declarations Data Cleaning and Wrangling

In this section, we clean and wrangle FEMA disaster declaration data to prepare it for analysis. The raw data is filtered to include only disasters from the past 10 years, joined with ZIP codes using a county-level crosswalk, and aggregated to generate a ZIP-level disaster count feature for downstream housing price modeling.

The final output table, `zip_disaster_counts`, contains the number of FEMA disaster declarations per ZIP code over the last decade. This will be used as a static risk feature in our housing price model.





##2.1 Import Required Libraries



In [66]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests

## 2.2 Load FEMA Disaster Declarations Data

Note: We loaded the full dataset here to understand the available columns. In the next step, we’ll select only the relevant ones.

In [67]:
fema_url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv"
fema_file_path = "DisasterDeclarationsSummaries.csv"

# Download the file if it doesn't already exist
if not os.path.exists(fema_file_path):
    response = requests.get(fema_url)
    with open(fema_file_path, "wb") as file:
        file.write(response.content)

# Load the entire dataset first (optional: you can limit rows for quick testing)
fema_raw_df = pd.read_csv(fema_file_path)

# Preview full column names
fema_raw_df.head()


<ipython-input-67-90417b007592>:11: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  fema_raw_df = pd.read_csv(fema_file_path)


,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,EM-3611-VI,3611,VI,EM,2024-08-18T00:00:00.000Z,2024,Tropical Storm,TROPICAL STORM ERNESTO,0,0,...,99010,St. Croix (Island) (County-equivalent),24126,NaN,2024080901,2,"4,M,W,Z",2025-03-25T22:41:27.147Z,4a8baf2d2f9117b4381e823fd96a41ea5f5c72d3,488c7bb4-5f20-4108-b499-99ef33953617
1,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,...,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,...,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
3,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,...,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,EM-3611-VI,3611,VI,EM,2024-08-18T00:00:00.000Z,2024,Tropical Storm,TROPICAL STORM ERNESTO,0,0,...,99020,St. John (Island) (County-equivalent),24126,NaN,2024080901,2,"4,M,W,Z",2025-03-25T22:41:27.147Z,049f9b3d480604687cd32784486d584b44381ff4,4576151c-0d18-478f-a342-1f89463a736e


## 2.3 Select Relevant Columns：

In [68]:
# Define the columns we want to keep
columns_to_use = ['disasterNumber', 'state', 'designatedArea', 'declarationDate']

# Select only relevant columns
fema_df = fema_raw_df[columns_to_use].copy()

# Drop rows with missing designated area (county-level location)
fema_df = fema_df.dropna(subset=['designatedArea'])

# Preview cleaned structure
fema_df.head()


,disasterNumber,state,designatedArea,declarationDate
0,3611,VI,St. Croix (Island) (County-equivalent),2024-08-18T00:00:00.000Z
1,5529,OR,Washington (County),2024-08-09T00:00:00.000Z
2,5528,OR,Jefferson (County),2024-08-06T00:00:00.000Z
3,5527,OR,Deschutes (County),2024-08-02T00:00:00.000Z
4,3611,VI,St. John (Island) (County-equivalent),2024-08-18T00:00:00.000Z


## 2.4 Filter Data from the Last 10 Years

In [69]:
# Convert declarationDate to timezone-aware datetime
fema_df['declarationDate'] = pd.to_datetime(fema_df['declarationDate'], errors='coerce', utc=True)

# Define the 10-year cutoff date (adjust as needed)
cutoff_date = pd.to_datetime("2013-01-01", utc=True)

# Filter the dataset for disasters declared from 2013 to present
fema_recent_df = fema_df[fema_df['declarationDate'] >= cutoff_date].copy()

# Check the date range
print("Earliest disaster:", fema_recent_df['declarationDate'].min())
print("Latest disaster:", fema_recent_df['declarationDate'].max())
print("Total records:", fema_recent_df.shape[0])

# Preview
fema_recent_df.head()


Earliest disaster: 2013-01-03 00:00:00+00:00
Latest disaster: 2025-04-05 00:00:00+00:00
Total records: 24988


,disasterNumber,state,designatedArea,declarationDate
0,3611,VI,St. Croix (Island) (County-equivalent),2024-08-18 00:00:00+00:00
1,5529,OR,Washington (County),2024-08-09 00:00:00+00:00
2,5528,OR,Jefferson (County),2024-08-06 00:00:00+00:00
3,5527,OR,Deschutes (County),2024-08-02 00:00:00+00:00
4,3611,VI,St. John (Island) (County-equivalent),2024-08-18 00:00:00+00:00


## 2.5 Clean FEMA Data (Standardize & Prepare for Join)

In [70]:
# Drop rows with missing designated area after filtering
fema_recent_df = fema_recent_df.dropna(subset=['designatedArea'])

# Remove any content inside parentheses, including the parentheses themselves
fema_recent_df['designatedArea'] = fema_recent_df['designatedArea'].str.replace(r"\s*\(.*?\)", "", regex=True)

# Standardize text fields: uppercase and strip whitespace
fema_recent_df['designatedArea'] = fema_recent_df['designatedArea'].str.upper().str.strip()
fema_recent_df['state'] = fema_recent_df['state'].str.upper().str.strip()

# Rename for consistency with other datasets
fema_recent_df.rename(columns={'designatedArea': 'county'}, inplace=True)

# Create a unique join key for ZIP code mapping
fema_recent_df['county_state'] = fema_recent_df['county'] + ", " + fema_recent_df['state']

# Final preview
print(fema_recent_df.shape)
fema_recent_df.drop_duplicates().head()


(24988, 5)


,disasterNumber,state,county,declarationDate,county_state
0,3611,VI,ST. CROIX,2024-08-18 00:00:00+00:00,"ST. CROIX, VI"
1,5529,OR,WASHINGTON,2024-08-09 00:00:00+00:00,"WASHINGTON, OR"
2,5528,OR,JEFFERSON,2024-08-06 00:00:00+00:00,"JEFFERSON, OR"
3,5527,OR,DESCHUTES,2024-08-02 00:00:00+00:00,"DESCHUTES, OR"
4,3611,VI,ST. JOHN,2024-08-18 00:00:00+00:00,"ST. JOHN, VI"


## 2.6 Load County-to-ZIP Mapping Table from Kaggle

In [71]:
# Install kagglehub if not already installed
!pip install kagglehub --quiet

import kagglehub
import os
import pandas as pd

# Download dataset from Kaggle
path = kagglehub.dataset_download("danofer/zipcodes-county-fips-crosswalk")

print("Path to dataset files:", path)

# List files
files = os.listdir(path)
print("Files in dataset:", files)

# Load the crosswalk CSV
file_path = os.path.join(path, "ZIP-COUNTY-FIPS_2017-06.csv")
zip_crosswalk_df = pd.read_csv(file_path)

# Preview
zip_crosswalk_df.head()


Path to dataset files: /root/.cache/kagglehub/datasets/danofer/zipcodes-county-fips-crosswalk/versions/1
Files in dataset: ['ZIP-COUNTY-FIPS_2017-06.csv']


,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


## 2.7 Preprocess ZIP Crosswalk for Joining with FEMA

In [72]:
# Clean up COUNTYNAME to match FEMA's 'county' format
def clean_county_name(name):
    name = name.upper().strip()
    name = name.replace(" COUNTY", "")
    name = name.replace(" PARISH", "")
    name = name.replace(" BOROUGH", "")
    name = name.replace(" CITY AND BOROUGH", "")
    name = name.replace(" MUNICIPALITY", "")
    return name

zip_crosswalk_df['COUNTYNAME'] = zip_crosswalk_df['COUNTYNAME'].apply(clean_county_name)
zip_crosswalk_df['STATE'] = zip_crosswalk_df['STATE'].str.upper().str.strip()

# Create county_state key for joining
zip_crosswalk_df['county_state'] = zip_crosswalk_df['COUNTYNAME'] + ", " + zip_crosswalk_df['STATE']

# Drop duplicates to keep 1 ZIP per county_state (one-to-many possible, we keep multiple for now)
zip_crosswalk_df = zip_crosswalk_df[['ZIP', 'county_state']].drop_duplicates()

# Preview
zip_crosswalk_df.head()


,ZIP,county_state
0,36003,"AUTAUGA, AL"
1,36006,"AUTAUGA, AL"
2,36067,"AUTAUGA, AL"
3,36066,"AUTAUGA, AL"
4,36703,"AUTAUGA, AL"


##  2.8 Merge FEMA with ZIP Codes


In [73]:
# Merge cleaned FEMA data with cleaned ZIP mapping
fema_with_zip = fema_recent_df.merge(zip_crosswalk_df, on='county_state', how='left')

# Check merge results
print(f"FEMA records before merge: {fema_recent_df.shape[0]}")
print(f"FEMA records after merge (with ZIP): {fema_with_zip['ZIP'].notna().sum()}")

# View some matched rows
fema_with_zip[['county', 'state', 'ZIP']].dropna().head()


FEMA records before merge: 24988
FEMA records after merge (with ZIP): 387209


,county,state,ZIP
1,WASHINGTON,OR,97078.0
2,WASHINGTON,OR,97133.0
3,WASHINGTON,OR,97008.0
4,WASHINGTON,OR,97225.0
5,WASHINGTON,OR,97075.0


## 2.9: Merge Validation — FEMA and ZIP Mapping

In this section, we validate whether the FEMA disaster records were successfully joined with ZIP codes using the county_state key. We test this across several dimensions: match rate, duplication, sample integrity, unmatched rows, and unmatched keys.

### 2.9.1 Match Rate Check
This checks whether the merge was successful for a significant portion of the data.

In [74]:
# Total unique FEMA records (pre-merge)
total_fema_disasters = fema_recent_df[['disasterNumber', 'county_state']].drop_duplicates().shape[0]

# How many unique FEMA records successfully got matched to at least one ZIP
matched_fema_disasters = fema_with_zip.dropna(subset=['ZIP'])[['disasterNumber', 'county_state']].drop_duplicates().shape[0]

# Match rate (now accurate)
match_rate = matched_fema_disasters / total_fema_disasters

print(f"Total unique FEMA disaster-county records: {total_fema_disasters}")
print(f"Matched to at least one ZIP: {matched_fema_disasters}")
print(f"Match rate: {match_rate:.2%}")


Total unique FEMA disaster-county records: 24904
Matched to at least one ZIP: 21888
Match rate: 87.89%


### 2.9.2 Duplication Check (County → Multiple ZIPs)

One FEMA disaster in a county can affect multiple ZIP codes. This step checks how many ZIPs are linked to each (disasterNumber + county_state) pair.

Summary of ZIP counts per FEMA record:

- Min: 1 ZIP
- Median: 11 ZIPs
- 75th percentile: 17 ZIPs
- Max: 494 ZIPs

Most counties were assigned to a reasonable number of ZIPs (under 20), though large counties like Los Angeles span many.


In [75]:
# One FEMA record (disasterNumber + county_state) can be duplicated if it matches multiple ZIPs
fema_dup_check = (
    fema_with_zip
    .groupby(['disasterNumber', 'county_state'])
    .size()
    .reset_index(name='zip_count')
)

# Check distribution of how many ZIPs each FEMA record was assigned to
fema_dup_check['zip_count'].describe()


,zip_count
count,24904.000000
mean,15.670093
std,25.043993
min,1.000000
25%,6.000000
50%,11.000000
75%,17.000000
max,494.000000


###2.9.3 Missing ZIPs (Rows That Didn't Match)
Identify FEMA records from counties not covered in the ZIP mapping dataset.

In [76]:
# FEMA rows that failed to match with any ZIP
missing_zip_df = fema_with_zip[fema_with_zip['ZIP'].isna()]

print(f"Number of FEMA records without ZIP after merge: {missing_zip_df.shape[0]}")
missing_zip_df[['county_state']].drop_duplicates().head(10)


Number of FEMA records without ZIP after merge: 3039


,county_state
0,"ST. CROIX, VI"
54,"ST. JOHN, VI"
55,"ST. THOMAS, VI"
56,"RESIGHINI RANCHERIA, CA"
121,"NEZ PERCE INDIAN RESERVATION, ID"
163,"COLVILLE INDIAN RESERVATION, WA"
196,"YAKAMA RESERVATION, WA"
450,"MESCALERO TRIBE, NM"
1280,"WASHINGTONMSA 5520,6480), RI"
2022,"YUKON-KOYUKUK, AK"


#### 2.9.3 ZIP Missing Analysis

After merging FEMA disaster data with ZIP codes, **3,039 records** could not be matched.

**Top unmatched `county_state` examples:**
- ST. CROIX, VI
- ST. JOHN, VI
- ST. THOMAS, VI
- MESCALERO TRIBE, NM
- NEZ PERCE INDIAN RESERVATION, ID
- YUKON-KOYUKUK, AK
- WASHINGTONMSA 5520,6480), RI

**Possible reasons:**
- U.S. Territories (e.g., VI) not included in ZIP mapping
- Indian Reservations not aligned with standard county names
- Remote or sparsely populated areas lacking ZIP assignments
- Irregular naming formats in FEMA data

**Action:** These records can be excluded


In [77]:
# Add a flag column to indicate whether ZIP was successfully matched
fema_with_zip['zip_matched'] = fema_with_zip['ZIP'].notna()

# Preview
fema_with_zip[['disasterNumber', 'county_state', 'ZIP', 'zip_matched']].head()


,disasterNumber,county_state,ZIP,zip_matched
0,3611,"ST. CROIX, VI",NaN,False
1,5529,"WASHINGTON, OR",97078.0,True
2,5529,"WASHINGTON, OR",97133.0,True
3,5529,"WASHINGTON, OR",97008.0,True
4,5529,"WASHINGTON, OR",97225.0,True


In [78]:
# Drop FEMA records that didn't match a ZIP
fema_matched_df = fema_with_zip[fema_with_zip['zip_matched']].copy()

# reset index for cleanliness
fema_matched_df.reset_index(drop=True, inplace=True)

# Preview
print(fema_matched_df.shape)
fema_matched_df[['disasterNumber', 'county_state', 'ZIP']].head()


(387209, 7)


,disasterNumber,county_state,ZIP
0,5529,"WASHINGTON, OR",97078.0
1,5529,"WASHINGTON, OR",97133.0
2,5529,"WASHINGTON, OR",97008.0
3,5529,"WASHINGTON, OR",97225.0
4,5529,"WASHINGTON, OR",97075.0


### 2.9.4 FEMA-ZIP Merge Summary

We compare the FEMA-ZIP dataset before and after filtering unmatched records:

| Metric                                  | Before Filtering (`fema_with_zip`) | After Filtering (`fema_matched_df`) |
|-----------------------------------------|-------------------------------------|--------------------------------------|
| Total rows                              | 390,248                             | 387,209                              |
| Records with matched ZIP                | 387,209                             | 387,209                              |
| Records with missing ZIP                | 3,039                               | 0                                    |
| Unique disaster-county combinations     | 24,904                              | 21,888                               |
| Match rate (disaster-county level)      | 87.89%                              | 100%                                 |
| Unique ZIP codes                        | 38,377                              | 38,377                               |

**Optimization Summary:**
- Removed 3,039 unmatched records (0.8%) from U.S. territories and special cases
- Ensured 100% ZIP code coverage for all retained FEMA disaster-county entries
- Final dataset `fema_matched_df` is clean, ZIP-aligned, and ready for ZIP-level aggregation


In [79]:
# Total rows
rows_before = fema_with_zip.shape[0]
rows_after = fema_matched_df.shape[0]

# Records with ZIP in the full merged dataset
matched_rows = fema_with_zip['ZIP'].notna().sum()
missing_rows = fema_with_zip['ZIP'].isna().sum()

# Unique disaster-county pairs
unique_disasters_before = fema_with_zip[['disasterNumber', 'county_state']].drop_duplicates().shape[0]
unique_disasters_after = fema_matched_df[['disasterNumber', 'county_state']].drop_duplicates().shape[0]

# Match rate (original dataset)
match_rate = unique_disasters_after / unique_disasters_before

# Match rate (final dataset): dynamically computed
matched_rows_final = fema_matched_df['ZIP'].notna().sum()
final_match_rate = matched_rows_final / fema_matched_df.shape[0]

# Unique ZIPs
unique_zips_before = fema_with_zip['ZIP'].nunique()
unique_zips_after = fema_matched_df['ZIP'].nunique()

# Print summary
print("FEMA-ZIP Merge Summary:")
print(f"Total rows (before):              {rows_before}")
print(f"Total rows (after):               {rows_after}")
print(f"Matched ZIPs:                     {matched_rows}")
print(f"Missing ZIPs:                     {missing_rows}")
print(f"Unique disaster-county (before):  {unique_disasters_before}")
print(f"Unique disaster-county (matched): {unique_disasters_after}")
print(f"Match rate (disaster-county):     {match_rate:.2%}")
print(f"Match rate (final dataset):       {final_match_rate:.2%}")
print(f"Unique ZIPs (before):             {unique_zips_before}")
print(f"Unique ZIPs (after):              {unique_zips_after}")


FEMA-ZIP Merge Summary:
Total rows (before):              390248
Total rows (after):               387209
Matched ZIPs:                     387209
Missing ZIPs:                     3039
Unique disaster-county (before):  24904
Unique disaster-county (matched): 21888
Match rate (disaster-county):     87.89%
Match rate (final dataset):       100.00%
Unique ZIPs (before):             38377
Unique ZIPs (after):              38377


##2.10 Disaster Count Aggregation by ZIP (Last 10 Years)

To support downstream analysis, we compute the number of FEMA disaster declarations in the past 10 years for each ZIP code. This gives a static risk exposure feature per ZIP that can be merged with housing data.

The aggregation is based on the cleaned FEMA dataset `fema_matched_df`, which includes only matched ZIP-level records between 2013 and 2025.

In [80]:
# Count the number of disasters per ZIP code
zip_disaster_counts = (
    fema_matched_df.groupby('ZIP')['disasterNumber']
    .nunique()  # count unique disasters per ZIP
    .reset_index(name='disaster_count_10yrs')
)

# Format ZIPs as 5-digit strings
zip_disaster_counts['ZIP'] = zip_disaster_counts['ZIP'].astype(int).astype(str).str.zfill(5)

# Reset index for cleanliness
zip_disaster_counts = zip_disaster_counts.reset_index(drop=True)

# Preview top ZIPs by disaster count
zip_disaster_counts.sort_values(by='disaster_count_10yrs', ascending=False).head(10)


,ZIP,disaster_count_10yrs
34992,91786,43
34978,91766,43
34943,91710,43
34942,91709,43
34681,90623,41
35617,93243,41
34683,90630,41
34684,90631,41
34688,90638,41
35754,93536,41


## Summary of Disaster Count Aggregation

The resulting table `zip_disaster_counts` contains the number of unique FEMA disaster declarations per ZIP code over the past 10 years (2013–2025). Each row represents a ZIP and its corresponding disaster exposure:

- **ZIP**: 5-digit ZIP code (string)
- **disaster_count_10yrs**: total number of FEMA disaster declarations in that ZIP from 2013 onward

This table will serve as a **static risk feature** that can be joined with the main housing dataset using the ZIP code as key. It helps capture regional environmental vulnerability, which may influence housing prices.
